Notebook for pickup density for an average day (Approach 1) 

In [1]:
from pyspark.ml import Pipeline
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.feature import VectorIndexer
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StringIndexer

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
3213,application_1513605045578_0452,pyspark,idle,Link,Link,✔


SparkSession available as 'spark'.


In [ ]:

# Load and parse the data file, converting it to a DataFrame.
data = spark.read.load("hdfs:///Projects/ID2223nyctaxi/prepared_taxirides/tiny_green_2013-10_no_geohash",format='com.databricks.spark.csv', header='true',inferSchema='true')
#print(data.take(5))
#data.show()
#print(data.take(5))

timecatIndexer = StringIndexer(inputCol="time_cat", outputCol="time_cat_in")
daycatIndexer = StringIndexer(inputCol="day_cat", outputCol="day_cat_in")
#pickupIndexer = StringIndexer(inputCol="pickup_location", outputCol="pickup_location_in")
#dropoffIndexer = StringIndexer(inputCol="dropoff_location", outputCol="dropoff_location_in")

data1 = timecatIndexer.fit(data).transform(data)
data2 = daycatIndexer.fit(data).transform(data1)
#data3 = pickupIndexer.fit(data).transform(data2)
#data4 = dropoffIndexer.fit(data).transform(data3)

dataprepper = VectorAssembler(inputCols=["year", "month", "day", "time_cat_in", "time_num", "time_cos", "time_sin","day_cat_in", "day_num", "day_cos","day_sin", "weekend", "pickup_latitude", "pickup_longitude"], outputCol="features")

#pipe = Pipeline(stages=[timecatIndexer,daycatIndexer,pickupIndexer, dropoffIndexer,dataprepper])
sdkfj = dataprepper.transform(data2).select("features")

print(sdkfj.take(5))
sdkfj.show(4)

#featureIndexer = VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=4)
#featureIndexer.transform(sdkfj).show()

# Automatically identify categorical features, and index them.
# Set maxCategories so features with > 4 distinct values are treated as continuous.

featureIndexer = VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=4).fit(sdkfj)

# Split the data into training and test sets (30% held out for testing)
(trainingData, testData) = data.randomSplit([0.8, 0.2])
print(trainingData.take(5))

# Train a RandomForest model.
rf = RandomForestRegressor(featuresCol="indexedFeatures").setMaxBins(48).setMaxDepth(10).setNumTrees(30)

# Chain indexer and forest in a Pipeline
pipeline = Pipeline(stages=[timecatIndexer,daycatIndexer, dataprepper, featureIndexer, rf])

# Train model.  This also runs the indexer.
model = pipeline.fit(trainingData)

# Make predictions.
predictions = model.transform(testData)

# Select example rows to display.
#predictions.sort("label", ascending = False).select("prediction", "label").write.format("com.databricks.spark.csv").option("header", "true").save("hdfs:///Projects/ID2223nyctaxi/predictions/tiny_green_2013-10_no_geohash_res2")


# Select (prediction, true label) and compute test error
evaluator = RegressionEvaluator(labelCol="label", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

rfModel = model.stages[4]
print(rfModel)  # summary only

#print(predictions.select("features").take(5))


[Row(features=DenseVector([2013.0, 10.0, 25.0, 5.0, 0.7188, -0.1951, -0.9808, 3.0, 0.6741, -0.459, -0.8884, 0.0, 40.5872, -73.9539])), Row(features=DenseVector([2013.0, 10.0, 23.0, 24.0, 0.4896, -0.9979, 0.0654, 2.0, 0.3557, -0.6162, 0.7876, 0.0, 40.5873, -73.9539])), Row(features=DenseVector([2013.0, 10.0, 25.0, 4.0, 0.7396, -0.0654, -0.9979, 3.0, 0.6771, -0.4423, -0.8969, 0.0, 40.5873, -73.954])), Row(features=DenseVector([2013.0, 10.0, 11.0, 33.0, 0.3229, -0.4423, 0.8969, 3.0, 0.6176, -0.7394, -0.6733, 0.0, 40.5843, -73.932])), Row(features=DenseVector([2013.0, 10.0, 20.0, 5.0, 0.7188, -0.1951, -0.9808, 5.0, 0.9598, 0.9683, -0.2498, 1.0, 40.682, -74.0073]))]
+--------------------+
|            features|
+--------------------+
|[2013.0,10.0,25.0...|
|[2013.0,10.0,23.0...|
|[2013.0,10.0,25.0...|
|[2013.0,10.0,11.0...|
+--------------------+
only showing top 4 rows

[Row(year=2013, month=10, day=1, time_cat=u'00:00', time_num=0.010416666666666666, time_cos=0.9978589232386035, time_sin=